# 🏍️ LossZero: Motorcycle Night Ride SegFormer-B2 Optimized

이 노트북은 **SegFormer-B2** 모델을 사용하여 야간 오토바이 주행 이미지의 시멘틱 세그멘테이션을 수행합니다.

### 🛠️ 주요 시나리오
- **모델**: SegFormer-B2 (Transformer 기반)
- **백본**: MiT-B2
- **사전 학습**: Cityscapes (도로 환경 특화)
- **최적화**: AdamW + FP16 Mixed Precision
- **손실 함수**: Weighted CrossEntropy (중요 객체 가중치 부여)

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import SegformerForSemanticSegmentation, SegformerConfig
from torch.amp import autocast, GradScaler
from tqdm.auto import tqdm

print(f"PyTorch version: {torch.__version__}")


## Colab 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [77]:
def get_device():
    if torch.cuda.is_available():
        return "cuda"

    return "cpu"

def num_worker():
    if torch.cuda.is_available():
        return os.cpu_count()

    return 0

# ⚙️ 설정 (Configuration)
#DATA_DIR = "/content/drive/MyDrive/motor_model"
DATA_DIR = os.path.expanduser("~/Projects/LossZero/data/Motorcycle Night Ride Dataset")
print("Detected Local Environment")

JSON_PATH = os.path.join(DATA_DIR, "COCO_motorcycle (pixel).json")
IMG_DIR = os.path.join(DATA_DIR, "images")

CFG = {
    "project": "LossZero",
    "model_name": "nvidia/segformer-b2-finetuned-cityscapes-1024-1024",
    "img_size": (352, 352),
    "batch_size": 4,
    "lr": 1e-4,
    "epochs": 25,
    "device": get_device(),
    "num_worker": num_worker()
}

print(f"Using device: {CFG['device']}")
print(f"Data directory: {DATA_DIR}")

Detected Local Environment
Using device: cpu
Data directory: /Users/jamesyang/Projects/LossZero/data/Motorcycle Night Ride Dataset


In [78]:
def create_mask_from_json(coco, img_id, img_info, id_to_idx):
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

    for ann in anns:
        cat_id = ann['category_id']
        if cat_id in id_to_idx:
            cls_idx = id_to_idx[cat_id]
            pixel_mask = coco.annToMask(ann)
            mask[pixel_mask == 1] = cls_idx

    return mask

def process_single_data(coco, img_id, img_dir, id_to_idx, transform=None):
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(img_dir, img_info['file_name'])

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    mask = create_mask_from_json(coco, img_id, img_info, id_to_idx)

    if transform:
        augmented = transform(image=image, mask=mask)
        image, mask = augmented['image'], augmented['mask']

    return image, torch.as_tensor(mask).long()

train_transform = A.Compose([
    #  원본 해상도에서 352x352 크기로 무작위 추출 (화질 저하 없음)
    A.RandomCrop(height=CFG['img_size'][0], width=CFG['img_size'][1], p=1.0),
    A.PadIfNeeded(min_height=CFG['img_size'][0], min_width=CFG['img_size'][1], p=1.0),

    # --- 야간 전용 Augmentation 추가 ---
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5), # 어두운 저조도 개선
    A.GaussNoise(std_range=(0.02, 0.05), p=0.3), # 야간 노이즈 대응

    # --- 기하학적 변형 (데이터 수 보충용) ---
    A.HorizontalFlip(p=0.5), # 좌우 반전
    # 0.0625는 머신러닝/딥러닝 커뮤니티에서 오랫동안 검증된 '사실상 표준(De Facto Standard)
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5), # 이동/크기/회전

    # ImageNet 데이타셋의 평균값 나쁘지 않음. SegFormer가 ImageNet/Cityscapes로 배웠으니까
    # 모델이 새로운 사진을 받을 때: 입력_이미지 = (원본_이미지 - 평균) / 표준편차
    # 이렇게 계산해주면, 어떤 사진이 들어와도 "평균이 0이고 표준편차가 1인(Standard Normal Distribution)" 아주 예쁜 데이터로 변신
    # 전체 야간 데이터셋의 Mean/Std를 직접 계산한 값
    A.Normalize(mean=(0.281, 0.268, 0.346), std=(0.347, 0.290, 0.292)),
    ToTensorV2()
])

coco = COCO(JSON_PATH)
img_ids = list(coco.imgs.keys())
cat_ids = coco.getCatIds()
id_to_idx = {cat_id: i for i, cat_id in enumerate(cat_ids)}
print(f"Category Mapping: {id_to_idx}")

/Users/jamesyang/.pyenv/versions/3.12.2/envs/aipel/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


loading annotations into memory...
Done (t=0.91s)
creating index...
index created!
Category Mapping: {1329681: 0, 1323885: 1, 1323884: 2, 1323882: 3, 1323881: 4, 1323880: 5}


## Traing / Val 분

In [80]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class MotorcycleNightRideDataset(Dataset):
    def __init__(self, coco, img_ids, img_dir, id_to_idx, transform=None):
        self.coco = coco
        self.img_ids = img_ids
        self.img_dir = img_dir
        self.id_to_idx = id_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        image, mask = process_single_data(self.coco, img_id, self.img_dir, self.id_to_idx, self.transform)
        return image, mask

# 1. 데이터 로드 및 ID 분할 (8:2)
coco = COCO(JSON_PATH)
all_ids = list(coco.imgs.keys())
train_ids, val_ids = train_test_split(all_ids, test_size=0.2, random_state=42)

# 2. Transform 정의 (기존 정의 활용 및 Val용 추가)
val_transform = A.Compose([
    A.Resize(CFG['img_size'][0], CFG['img_size'][1]),
    A.Normalize(mean=(0.281, 0.268, 0.346), std=(0.347, 0.290, 0.292)),
    ToTensorV2()
])

# 3. 데이터셋 인스턴스 생성
train_dataset = MotorcycleNightRideDataset(coco, train_ids, IMG_DIR, id_to_idx, train_transform)
val_dataset = MotorcycleNightRideDataset(coco, val_ids, IMG_DIR, id_to_idx, val_transform)

# 4. 데이터 로더 생성
train_loader = DataLoader(
    train_dataset, 
    batch_size=CFG['batch_size'], 
    shuffle=True, 
    num_workers=CFG['num_worker'],
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=CFG['batch_size'], 
    shuffle=False, 
    num_workers=CFG['num_worker'],
    pin_memory=True
)

print(f"✅ Data Ready: Train={len(train_ids)}, Val={len(val_ids)}")

loading annotations into memory...
Done (t=0.85s)
creating index...
index created!
✅ Data Ready: Train=160, Val=40


### 📉 클래스별 분포 요약 (내림차순)

1. **Undrivable (주행 불가 영역)**: **42.9%** (압도적 1위)
   - 배경(하늘, 건물, 풀숲 등)이 이미지의 절반 가까이 차지합니다.
2. **Road (주행 가능 도로)**: **27.1%**
   - 도로 자체도 꽤 많은 영역을 차지합니다.
3. **My bike (내 오토바이)**: **15.8%**
   - 주행자 시점이라 내 오토바이가 항상 보이기 때문에 비율이 높습니다.
4. **Rider (탑승자)**: **8.1%**
   - 다른 오토바이 운전자나 내 신체가 포함된 것으로 보입니다.
5. **Moveable (이동 물체)**: **4.7%**
   - 다른 차량, 보행자 등 안전에 가장 중요한 장애물인데 비율이 매우 낮습니다.
6. **Lane Mark (차선)**: **1.4%**
   - 가장 심각한 불균형입니다. 도로 주행의 핵심인 차선이 고작 1% 남짓입니다.

In [81]:
id2label = {i: coco.loadCats(cat_id)[0]['name'] for cat_id, i in id_to_idx.items()}
label2id = {v: k for k, v in id2label.items()}

model = SegformerForSemanticSegmentation.from_pretrained(
    CFG['model_name'],
    num_labels=len(id_to_idx),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
).to(CFG['device'])

optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['lr'], weight_decay=0.01)

# ⚖️ 클래스별 가중치 설정 (Class Weights)
weights = torch.tensor([
    5.0,   # Rider (8.1%) -> 적당히 높임
    2.0,   # My bike (15.8%) -> 낮춤 (이미 많음)
    10.0,  # Moveable (4.7%) -> 강력하게 높임
    20.0,  # Lane Mark (1.4%) -> 아주 강력하게!! (핵심)
    1.0,   # Road (27.1%) -> 기본
    0.5    # Undrivable (42.9%) -> 낮춤 (너무 많아서 방해됨)
], dtype=torch.float).to(CFG['device'])

criterion = nn.CrossEntropyLoss(weight=weights)

scaler = GradScaler('cuda') if CFG['device'] == 'cuda' else None

Loading weights:   0%|          | 0/380 [00:00<?, ?it/s]

SegformerForSemanticSegmentation LOAD REPORT from: nvidia/segformer-b2-finetuned-cityscapes-1024-1024
Key                           | Status   |                                                                                                    
------------------------------+----------+----------------------------------------------------------------------------------------------------
decode_head.classifier.bias   | MISMATCH | Reinit due to size mismatch - ckpt: torch.Size([19]) vs model:torch.Size([6])                      
decode_head.classifier.weight | MISMATCH | Reinit due to size mismatch - ckpt: torch.Size([19, 768, 1, 1]) vs model:torch.Size([6, 768, 1, 1])

Notes:
- MISMATCH	:ckpt weights were loaded, but they did not match the original empty weight shapes.


In [ ]:
print("🚀 SegFormer-B2 Training Start...")

for epoch in range(CFG['epochs']):
    # --- Training Phase ---
    model.train()
    train_loss_sum = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]")
    
    for images, masks in pbar:
        X = images.to(CFG['device']).contiguous()
        y = masks.to(CFG['device']).contiguous()
        
        optimizer.zero_grad()
        
        # Mixed Precision 지원 (CUDA 전용)
        if CFG['device'] == 'cuda' and scaler:
            with torch.amp.autocast('cuda'):
                outputs = model(X).logits
                upsampled_logits = nn.functional.interpolate(outputs, size=y.shape[-2:], mode="bilinear", align_corners=False)
                loss = criterion(upsampled_logits, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(X).logits
            upsampled_logits = nn.functional.interpolate(outputs, size=y.shape[-2:], mode="bilinear", align_corners=False)
            loss = criterion(upsampled_logits, y)
            loss.backward()
            optimizer.step()
            
        train_loss_sum += loss.item()
        pbar.set_postfix(Loss=f"{loss.item():.4f}")
    
    avg_train_loss = train_loss_sum / len(train_loader)

    # --- Validation Phase ---
    model.eval()
    val_loss_sum = 0
    with torch.no_grad():
        for images, masks in val_loader:
            X = images.to(CFG['device']).contiguous()
            y = masks.to(CFG['device']).contiguous()
            
            outputs = model(X).logits
            upsampled_logits = nn.functional.interpolate(outputs, size=y.shape[-2:], mode="bilinear", align_corners=False)
            loss = criterion(upsampled_logits, y)
            val_loss_sum += loss.item()
            
    avg_val_loss = val_loss_sum / len(val_loader)
    print(f"📝 Epoch [{epoch+1}/{CFG['epochs']}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

🚀 SegFormer-B2 Training Start...


Epoch 1 [Train]:   0%|          | 0/40 [00:00<?, ?it/s]

📝 Epoch [1/25] Train Loss: 1.6318 | Val Loss: 1.5289


Epoch 2 [Train]:   0%|          | 0/40 [00:00<?, ?it/s]

### 🛠️ 주요 평가 항목
- **mIoU** (Mean Intersection over Union):
  - **Category-specific IoU**
  - **Boundary IoU**
- **실시간성 및 하드웨어 지표**
  - **Model Parameters**
  - **MACs** Multiply-Accumulate Operations
    - y = wx + b 에서 wx + b를 1MAC 이라고 한다.
  - **GFLOPs** Giga Floating Point Operations
    -  모델을 한 번 실행(Forward Pass)할 때 필요한 총 부동 소수점 연산량
    - 보통 1MAC = 2FLOPs
  - **Average Inference Latency**
  - **Frames Per Second (FPS)**
- **Safety-critical Metrics**

- TODO
  - 테스트 이미지로 평가할 것 ( 현재는 전체 이미지 SET 사용 )

In [ ]:
import torch
import numpy as np
import cv2
from sklearn.metrics import confusion_matrix

def compute_category_iou(preds, targets, num_classes):
    """
    Confusion Matrix를 사용하여 클래스별 IoU(Intersection over Union)를 계산합니다.
    Args:
        preds (torch.Tensor): 예측된 마스크 (N, H, W).
        targets (torch.Tensor): 실제 정답(Ground truth) 마스크 (N, H, W).
        num_classes (int): 클래스 개수.
    Returns:
        np.ndarray: 각 클래스별 IoU 값.
    """
    preds_flat = preds.flatten().cpu().numpy()
    targets_flat = targets.flatten().cpu().numpy()

    # 유효하지 않은 타겟 값 제외 (예: 패딩 또는 ignore_index가 포함된 경우)
    valid_mask = (targets_flat >= 0) & (targets_flat < num_classes)
    preds_flat = preds_flat[valid_mask]
    targets_flat = targets_flat[valid_mask]

    # 혼동 행렬(Confusion Matrix) 계산
    cm = confusion_matrix(targets_flat, preds_flat, labels=range(num_classes))

    # 클래스별 IoU 계산 공식 = TP / (TP + FP + FN)
    intersection = np.diag(cm)
    ground_truth_set = cm.sum(axis=1)
    predicted_set = cm.sum(axis=0)
    union = ground_truth_set + predicted_set - intersection

    # 0으로 나누는 오류(division by zero) 방지
    iou = intersection / (union + 1e-6)
    return iou

def get_boundary(mask, dilation_pixels=2):
    """
    이진 마스크(binary mask)에서 경계선 영역을 추출합니다.
    """
    mask = mask.astype(np.uint8)
    kernel = np.ones((3, 3), np.uint8)
    # 침식 연산(Erosion)을 통해 내부를 깎아냄
    eroded = cv2.erode(mask, kernel, iterations=dilation_pixels)
    # 원본 마스크에서 침식된 마스크를 빼서 경계선만 남김
    boundary = mask - eroded
    return boundary

def compute_boundary_iou(preds, targets, num_classes, dilation_pixels=2):
    """
    각 클래스별로 Boundary IoU를 계산합니다.
    """
    preds_np = preds.cpu().numpy()
    targets_np = targets.cpu().numpy()
    b_ious = []

    for c in range(num_classes):
        class_preds = (preds_np == c)
        class_targets = (targets_np == c)

        ious_per_batch = []
        for i in range(preds_np.shape[0]): # 배치 내 각 이미지별로 반복
            gt_boundary = get_boundary(class_targets[i], dilation_pixels)
            pred_boundary = get_boundary(class_preds[i], dilation_pixels)

            intersection = ((gt_boundary > 0) & (pred_boundary > 0)).sum()
            union = ((gt_boundary > 0) | (pred_boundary > 0)).sum()

            if union == 0:
                # 정답(GT)과 예측 모두 해당 클래스의 경계선이 없는 경우, 
                # 완벽하게 일치하는 것으로 간주하여 IoU 1.0 부여
                # (특정 클래스가 GT에 없을 때 전체 평균에 미치는 영향을 고려한 처리)
                ious_per_batch.append(1.0)
            else:
                ious_per_batch.append(intersection / union)

        # 배치 내의 어떤 이미지에서도 해당 클래스의 경계선이 발견되지 않은 경우
        if not ious_per_batch:
            b_ious.append(0.0) # 필요에 따라 NaN 등으로 처리 가능
        else:
            b_ious.append(np.mean(ious_per_batch))

    return np.array(b_ious)

In [ ]:
print("🚀 SegFormer-B2 Evaluation Start...")

model.eval()

all_preds = []
all_targets = []

total_category_ious = np.zeros(CFG_EVAL['num_classes'])
total_boundary_ious = np.zeros(CFG_EVAL['num_classes'])
num_batches = 0

with torch.no_grad():
    eval_pbar = tqdm(range(0, len(img_ids), CFG['batch_size']), desc="Evaluating")
    for i in eval_pbar:
        batch_ids = img_ids[i : i + CFG['batch_size']]

        images, masks = [], []
        for img_id in batch_ids:
            img, msk = process_single_data(coco, img_id, IMG_DIR, id_to_idx, train_transform)
            images.append(img)
            masks.append(msk)

        X = torch.stack(images).to(CFG['device']).contiguous()
        y = torch.stack(masks).to(CFG['device']).contiguous()

        outputs = model(X).logits.contiguous()
        upsampled_logits = nn.functional.interpolate(
            outputs,
            size=y.shape[-2:],
            mode="bilinear",
            align_corners=False
        ).contiguous()

        preds = upsampled_logits.argmax(dim=1)

        # 해당 batch의 IoU 계산 
        category_ious = compute_category_iou(preds, y, CFG_EVAL['num_classes'])
        boundary_ious = compute_boundary_iou(preds, y, CFG_EVAL['num_classes'])

        total_category_ious += category_ious
        total_boundary_ious += boundary_ious
        num_batches += 1

# 평균 IoUs 계산
mean_category_ious = total_category_ious / num_batches
mean_boundary_ious = total_boundary_ious / num_batches

print()
print("--- Evaluation Results ---")
print("Category-specific IoU (Mean over batches):")
for i, val in enumerate(mean_category_ious):
    print(f"  Class {i} ({id2label[i]}): {val:.4f}")

print()
print("Boundary IoU (Mean over batches):")
for i, val in enumerate(mean_boundary_ious):
    print(f"  Class {i} ({id2label[i]}): {val:.4f}")

print()
print("Mean Category IoU (mIoU):", np.mean(mean_category_ious))
print("Mean Boundary IoU (mBoU):", np.mean(mean_boundary_ious))

In [ ]:
# TODO : requirement.txt로 보내자
import sys
!{sys.executable} -m pip install thop

print("thop installed successfully.")

In [ ]:
import time
from thop import profile

print("🚀 Starting Model Profiling...")

# 2. 모델을 위한 더미 입력 텐서 생성
dummy_input = torch.randn(1, 3, CFG['img_size'][0], CFG['img_size'][1]).to(CFG['device'])

# 3. 더미 입력을 사용하여 모델의 MACs 및 매개변수(params)를 계산하기 위해 thop.profile 사용. verbose=False로 설정.
macs, params = profile(model, inputs=(dummy_input,), verbose=False)

# 4. 계산된 MACs를 1e9로 나누어 GFLOPs로 변환.
gflops = macs / 1e9

print(f"Model Parameters (M): {params / 1e6:.2f}")
print(f"MACs (G): {macs / 1e9:.2f}")
print(f"GFLOPs: {gflops:.2f}")

# 5. 지연 시간 및 FPS 측정을 위한 변수 초기화.
num_warmup_runs = 10
num_inference_runs = 100
total_latency = 0.0

# 모델을 평가 모드로 설정
model.eval()

# 6. 워밍업 실행 수행
print(f"Performing {num_warmup_runs} warm-up runs...")
with torch.no_grad():
    for _ in range(num_warmup_runs):
        _ = model(dummy_input)

# 7. 추론 지연 시간 측정
print(f"Measuring latency over {num_inference_runs} inference runs...")
with torch.no_grad():
    for _ in range(num_inference_runs):
        start_time = time.perf_counter()
        _ = model(dummy_input)
        end_time = time.perf_counter()
        total_latency += (end_time - start_time)

# 8. 평균 지연 시간(ms) 계산
average_latency_ms = (total_latency / num_inference_runs) * 1000

# 9. FPS 계산
fps = 1000 / average_latency_ms

print(f"Average Inference Latency: {average_latency_ms:.2f} ms")
print(f"Frames Per Second (FPS): {fps:.2f}")
print("Model Profiling Complete.")